# Solución Analítica del Oscilador Armónico Cuántico

Para validar los resultados obtenidos mediante el método numérico de Numerov, utilizamos la solución exacta de la ecuación de Schrödinger para el potencial del oscilador armónico:

$$
V(x) = \frac{1}{2} m \omega^2 x^2
$$

En nuestro sistema de unidades (donde $m = 1$ y $\omega = 1$), la ecuación se reduce a:

$$
-\frac{1}{2} \frac{d^2 \psi}{dx^2} + \frac{1}{2} x^2 \psi = E \psi
$$

---

## 1. Energías Propias (Autovalores)

La teoría predice que la energía está cuantizada en niveles equiespaciados:

$$
E_n = n + \frac{1}{2}, \quad \text{para } n = 0, 1, 2, \dots
$$

| Estado ($n$)              | Energía ($E_n$) |
|----------------------------|------------------|
| 0 (Fundamental)            | 0.5              |
| 1 (Primer excitado)        | 1.5              |
| 2 (Segundo excitado)       | 2.5              |

---

## 2. Funciones de Onda (Autovectores)

Las soluciones analíticas vienen dadas por los Polinomios de Hermite ($H_n$) multiplicados por una envolvente gaussiana:

$$
\psi_n(x) = \frac{1}{\sqrt{2^n n!}} 
\left( \frac{1}{\pi} \right)^{1/4} 
e^{-x^2/2} H_n(x)
$$

Esta solución es la referencia ideal para medir el error de truncamiento de nuestro algoritmo de Numerov.

In [1]:
import numpy as np
from scipy.special import hermite
from math import factorial

def solucion_analitica_oscilador(n_estados=6, Rmin=-6, Rmax=6, N=1000):
    x = np.linspace(Rmin, Rmax, N)
    h = x[1] - x[0]
    
    print(f"Los primeros {n_estados} autovalores analíticos:")
    
    # 1. Calcular Energías Exactas
    energias = [n + 0.5 for n in range(n_estados)]
    
    # Formatear salida de autovalores (estilo Fortran)
    header = f"{N:4d}  " + "  ".join([f"{E:9.6f}" for E in energias])
    print(header)
    
    # 2. Calcular Densidades de Probabilidad |psi|^2
    # La constante (1/pi)^(1/4) normaliza la gaussiana
    norm_const = (1.0 / np.pi)**0.25
    
    densidades = np.zeros((N, n_estados))
    
    for n in range(n_estados):
        # Polinomio de Hermite de orden n
        Hn = hermite(n)
        
        # Factor de normalización del estado n
        prefactor = 1.0 / np.sqrt(2**n * factorial(n))
        
        # Función de onda psi_n(x)
        psi = prefactor * norm_const * np.exp(-x**2 / 2.0) * Hn(x)
        
        # Densidad de probabilidad
        densidades[:, n] = psi**2

    # 3. Guardar resultados en el formato solicitado
    with open("data_osci_analitica.txt", "w") as f:
        # Cabecera con N y energías
        f.write(f"#{N//2:4d}      " + " ".join([f"{E:10.6f}" for E in energias]) + "\n")
        
        for i in range(N):
            linea_densidades = " ".join([f"{densidades[i, j]:10.5f}" for j in range(n_estados)])
            f.write(f"{x[i]:10.5f} {linea_densidades}\n")
            
    print(f"\nResultados analíticos guardados en 'data_osci_analitica.txt'")
    return x, densidades

# Ejecución
x_an, dens_an = solucion_analitica_oscilador()

Los primeros 6 autovalores analíticos:
1000   0.500000   1.500000   2.500000   3.500000   4.500000   5.500000

Resultados analíticos guardados en 'data_osci_analitica.txt'


# Solución Analítica del Pozo Lineal $V(x) = |x|$

El potencial de valor absoluto, definido como $V(x) = |x|$, presenta un desafío interesante debido a la discontinuidad en la derivada del potencial en $x = 0$. La ecuación de Schrödinger a resolver es:

$$
-\frac{1}{2} \frac{d^2 \psi}{dx^2} + |x| \psi = E \psi
$$

---

## 1. Las Funciones de Airy

Para $x > 0$, la ecuación se puede transformar en la ecuación diferencial de Airy. Las soluciones que no divergen en el infinito son las funciones de Airy de primera clase, $Ai(z)$.

La solución general se construye considerando la simetría del potencial:

- **Estados pares:** La derivada de la función de onda debe ser cero en el origen:
  
  $$
  \psi'(0) = 0
  $$

- **Estados impares:** La función de onda debe ser cero en el origen:
  
  $$
  \psi(0) = 0
  $$

---

## 2. Autovalores de Energía

Las energías propias $E_n$ se obtienen a partir de los ceros (raíces) de la función de Airy o de su derivada:

- Para **estados impares** ($n = 1, 3, 5, \dots$):  
  Las energías dependen de las raíces negativas de $Ai(z)$, llamadas $a_k$.

- Para **estados pares** ($n = 0, 2, 4, \dots$):  
  Las energías dependen de las raíces negativas de $Ai'(z)$, llamadas $a'_k$.

La relación matemática es:

$$
E_n = -\frac{a_k}{2^{1/3}}
\quad \text{o} \quad
E_n = -\frac{a'_k}{2^{1/3}}
$$

A diferencia del oscilador armónico, los niveles de energía no están equiespaciados; la distancia entre ellos disminuye a medida que $n$ aumenta.

In [2]:
import numpy as np
from scipy.special import ai_zeros, airy

def solucion_analitica_lineal(n_estados=6, Rmin=-10, Rmax=10, N=1600):
    x = np.linspace(Rmin, Rmax, N)
    
    # El potencial |x| es simétrico, calculamos raíces de Ai y Ai'
    # ai_zeros(n) devuelve (raíces de Ai, raíces de Ai', valores de Ai en raíces Ai', valores de Ai' en raíces Ai)
    ai_roots, aip_roots, _, _ = ai_zeros(n_estados)
    
    # Las energías se intercalan según la paridad
    # n=0 (par), n=1 (impar), n=2 (par)...
    energias = []
    p_idx, i_idx = 0, 0
    for n in range(n_estados):
        if n % 2 == 0: # Estado par (raíces de Ai')
            E = -aip_roots[p_idx] / (2**(1/3))
            p_idx += 1
        else:          # Estado impar (raíces de Ai)
            E = -ai_roots[i_idx] / (2**(1/3))
            i_idx += 1
        energias.append(E)

    print("--- COMPARATIVA ANALÍTICA (|x|) ---")
    print(f"Puntos: {N}")
    for n, E in enumerate(energias):
        paridad = "Par" if n % 2 == 0 else "Impar"
        print(f"n = {n} ({paridad}) | E = {E:9.6f}")

    # Cálculo de las funciones de onda normalizadas
    densidades = np.zeros((N, n_estados))
    for n in range(n_estados):
        E = energias[n]
        # Variable transformada para la función de Airy: z = 2^(1/3) * (x - E)
        # Para x > 0:
        z = (2**(1/3)) * (np.abs(x) - E)
        ai, aip, bi, bip = airy(z)
        
        psi = ai
        # Aplicamos la paridad manualmente para x < 0
        if n % 2 != 0: # Si es impar, invertimos el signo en x < 0
            psi = np.where(x < 0, -psi, psi)
            
        # Normalización numérica
        norma = np.sqrt(np.sum(psi**2) * (x[1]-x[0]))
        densidades[:, n] = (psi / norma)**2

    # Guardar en archivo
    nombre_archivo = "data_abs_analitica.txt"
    with open(nombre_archivo, "w") as f:
        header = " ".join([f"{E:10.6f}" for E in energias])
        f.write(f"#{N//2:4d}      {header}\n")
        for i in range(N):
            linea = " ".join([f"{densidades[i, j]:10.5f}" for j in range(n_estados)])
            f.write(f"{x[i]:10.5f} {linea}\n")

    print(f"\n[OK] Datos exactos guardados en '{nombre_archivo}'")
    return x, energias

# Ejecución
x_an, e_an = solucion_analitica_lineal()

--- COMPARATIVA ANALÍTICA (|x|) ---
Puntos: 1600
n = 0 (Par) | E =  0.808617
n = 1 (Impar) | E =  1.855757
n = 2 (Par) | E =  2.578096
n = 3 (Impar) | E =  3.244608
n = 4 (Par) | E =  3.825715
n = 5 (Impar) | E =  4.381671

[OK] Datos exactos guardados en 'data_abs_analitica.txt'
